In [1]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy.stats import bernoulli

# The Upper Confidence Bound (UCB)

In [2]:
class RestaurantThompsonSampler():
    def __init__(self, p):
        self.p = p
        self.estimated_p = 0
        self.N = 0
        
    def ucb(self,mean, n, nj):
        return mean + np.sqrt(2*np.log(n) / nj)
    
    def get_reward_from_true_distribution(self):
        s = bernoulli.rvs(self.p, size=1)[0]
        return s
    
    def update_current_distribution(self,x):
        self.N += 1.
        self.estimated_p = ((self.N - 1)*self.estimated_p + x) / self.N


In [3]:
def draw_distributions(R,i):
    for r in R:
        samps = bernoulli.rvs(r.estimated_p, size=100)
        sns.kdeplot(samps, fill=True)
    plt.title('Iteration %s'%(i+1), fontsize=20)
    plt.legend(['mu=%s'%(r.p) for r in R], fontsize=16)
    plt.xlim(-3,3)
    plt.xlabel('Average Satisfacton', fontsize=20)
    plt.ylabel('Density', fontsize=20)
    
    plt.show()

In [4]:
num_restaurants = 4
spacing = 1
R = [RestaurantThompsonSampler(i*0.2) for i in range(1, num_restaurants+1)]

# Experiment once (to not nj==0) 
for r in R :
    s = r.get_reward_from_true_distribution()
    r.update_current_distribution(s)

for i in range(2,1000):    
    #get a sample of the estimated mean (estimated p)
    post_samps = [r.ucb(r.estimated_p,i,r.N) for r in R]
    
    #index of distribution with highest ucb
    chosen_idx = post_samps.index(max(post_samps))
    
    #get a new sample from that distribution
    s = R[chosen_idx].get_reward_from_true_distribution()
    
    #update that distributions posterior
    R[chosen_idx].update_current_distribution(s)
    
    
for r in R :
    print('Real P = ', r.p, 'Estimated P = ',r.estimated_p)

Real P =  0.2 Estimated P =  0.21428571428571427
Real P =  0.4 Estimated P =  0.36956521739130443
Real P =  0.6000000000000001 Estimated P =  0.5762711864406777
Real P =  0.8 Estimated P =  0.7938271604938268
